In [6]:
!pip install -q wandb tensorflow==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorstore 0.1.71 requires ml_dtypes>=0.3.1, but you have ml-dtypes 0.2.0 which is incompatible.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.15.0 which is incompatible.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.15.0 which is incompatible.


In [7]:
# %% @title Environment Setup - Regression
import tensorflow as tf
from tensorflow.keras import layers, models
import wandb
from wandb.integration.keras import WandbCallback
import numpy as np
import matplotlib.pyplot as plt


# %%
insurance_df = pd.read_csv("https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv")
X = pd.get_dummies(insurance_df.drop("charges", axis=1))
y = insurance_df["charges"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled  = scaler.transform(X_test)




# %%
def create_regression_model():
    model = models.Sequential(name="sequential_reg")
    model.add(layers.Dense(128, activation="relu", input_shape=(X_train_scaled.shape[1],), name="dense_1"))
    model.add(layers.Dropout(0.3, name="dropout_1"))
    model.add(layers.Dense(64, activation="relu", name="dense_2"))
    model.add(layers.Dense(1, name="dense_3"))
    return model

reg_model = create_regression_model()
reg_model.summary()




# %%
detailed_reg_summary = tf.keras.utils.model_to_dot(reg_model, show_shapes=True).create(prog='dot', format='plain')
print("Detailed Regression Model Summary:")
print(detailed_reg_summary)




# %%
wandb.init(project="medical-regression", reinit=True)
wandb.config.update({"optimizer": "adam", "epochs": 100, "batch_size": 32})
reg_model.compile(optimizer="adam", loss="mse", metrics=["mae"])
history_reg = reg_model.fit(X_train_scaled, y_train,
                            validation_split=0.2,
                            epochs=100,
                            batch_size=32,
                            callbacks=[WandbCallback()])




# %%
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_pred_reg = reg_model.predict(X_test_scaled).flatten()
plt.figure(figsize=(8,6))
plt.scatter(y_test, y_test - y_pred_reg, alpha=0.5)
plt.axhline(0, color="red", linestyle="--")
plt.xlabel("True Charges")
plt.ylabel("Residuals")
plt.title("Residuals Plot")
plt.savefig("residuals_plot.png")
plt.show()
wandb.log({"residuals_plot": wandb.Image("residuals_plot.png")})

mse_val = mean_squared_error(y_test, y_pred_reg)
mae_val = mean_absolute_error(y_test, y_pred_reg)
r2_val  = r2_score(y_test, y_pred_reg)
wandb.log({"mse": mse_val, "mae": mae_val, "r2": r2_val})



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_reg"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_1 (Dense)                      │ (None, 128)                 │           1,536 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,857 (38.50 KB)

 Trainable params: 9,857 (38.50 KB)

 Non-trainable params: 0 (0.00 B)

Detailed Regression Model Summary:
b'graph 1 5.0694 6.1667\nnode 140157949294096 2.5347 5.5833 4.9722 1.1667 <<table border="0" cellborder="1" bgcolor="black" cellpadding="10"><tr><td colspan="2" bgcolor="black"><font point-size="16" color="white"><b>dense_1</b> (Dense)</font></td></tr><tr><td bgcolor="white"><font point-size="14">Input shape: <b>(None, 11)</b></font></td><td bgcolor="white"><font point-size="14">Output shape: <b>(None, 128)</b></font></td></tr></table>> solid record black lightgrey\nnode 140157967725008 2.5347 3.9167 5.0694 1.1667 <<table border="0" cellborder="1" bgcolor="black" cellpadding="10"><tr><td colspan="2" bgcolor="black"><font point-size="16" color="white"><b>dropout_1</b> (Dropout)</font></td></tr><tr><td bgcolor="white"><font point-size="14">Input shape: <b>(None, 128)</b></font></td><td bgcolor="white"><font point-size="14">Output shape: <b>(None, 128)</b></font></td></tr></table>> solid record black lightgrey\nnode 140157887372560 2.5347 2.25 4.9722 1.1

Epoch 1/100


AttributeError: 'Node' object has no attribute 'inbound_layers'